# about decorator

In [1]:
from collections import defaultdict
import time
from functools import wraps

In [2]:
"""1. decorator是应用包装函数的快捷方式。这有助于反复使用相同代码“包装”功能。减少重复！！！
   2. 但是装饰器会重写使用装饰器的函数的名字和注释文档，所以我们需要使用functools.wraps
   3. 装饰器并不是核心功能，"""
def func_1(n):
    for i in range(n):
        print(n)

        
"""python面向函数--->面向：如果一个东西面向某物，指的是：
   1.面向的这个东西可以作为参数（arg）传入；2. 这个东西可以作为变量赋值 3.可以作为一个return"""
def call_time(func_1, arg):
    start = time.time()
    func_1(arg)
    print('used time: {}'.format(time.time() - start))
    
    
function_called_time = defaultdict(int)

"""wraps 接受一个函数来进行装饰，并加入了复制函数名称、注释文档、参数列表等功能。
   这可以让我们在装饰器里面访问在装饰器之前的函数的属性
"""
def get_call_time(func):
    @wraps(func)
    def _inner(arg):   # 真正写参数的时候 *args, **kwargs ==> 表示可支持多个参数
        """it's inner function"""
        global function_called_time
        function_called_time[func.__name__] += 1  # 看一个函数调用多少次
        result = func(arg)
        print('function called time is：{}'.format(function_called_time[func.__name__]))
        return result
    return _inner  # 返回一个包装函数，这个包装函数的参数传给func作为参数

@get_call_time
def func_slow(n):
    for i in range(n):
        time.sleep(0.2)
        print(n)

# Dynamic Programming For Cutting Problems

In [3]:
from collections import defaultdict
import time
from functools import wraps

In [4]:
original_price = [1,5,8,9,10,17,17,20,24,30]
price = defaultdict(int)

for i, p in enumerate(original_price):
    price[i+1] = p
assert price[1] == 1

In [5]:
solution = {}
## for a given length N, we set the corresponding split parts
## solution = {
#     4: (2, 2)
# }

In [6]:

"""写一个装饰器，避免重复计算，也就是把已经计算过的东西存起来"""
def memo(func):
    cache = {}
    @wraps(func)
    def _wrap(n):
        if n in cache: result = cache[n]
        else:
            result = func(n)
            cache[n] = result
        return result
    return  _wrap


"""要注意递归的出口在哪里，不然就出不来了"""
"""注意如何把问题抽象，抽象出来之后其实就会变得比较简单"""
@memo
def r(n):
    max_price, split_point = max(
       #  [p[n], r(1) + r(n-1), r(2)+ r(n-2), ...r(n-1) + r(1)]
        [(price[n], 0)] + [(r(i) + r(n-i), i) for i in range(1, n)], key=lambda x: x[0]   
    )
    solution[n] = (split_point, n - split_point)
    return max_price


def not_cut(split): return split == 0  # 这是一个布尔值

# 构建solution，把要切割的目标值和上面得到的solution传入。然后得到一个将目标值分为几段的列表
def parse_solution(target_length, revenue_solution):
    left, right = revenue_solution[target_length]
    if not_cut(left):  # 如果切割后左边仍等于0，相当于没有切割，所以就不用在切割了，直接返回结果就可以了
        return [right]
    return parse_solution(left, revenue_solution) + parse_solution(right, revenue_solution)


In [8]:
r(231)

691

In [12]:
parse_solution(19,solution)

[3, 6, 10]

# Edit Distance

In [13]:
from functools import lru_cache

In [50]:
solution = {}

In [64]:
@lru_cache(maxsize=2**10)
def edit_distance(string1, string2):

    if len(string1) == 0: return len(string1)
    if len(string2) == 0: return len(string2)

    tail_s1 = string1[-1]
    tail_s2 = string2[-1]

    return min([
            edit_distance(string1[:-1], string2) + 1,
            edit_distance(string1, string2[:-1]) + 1,
            edit_distance(string1[:-1], string2[:-1]) + (0 if tail_s1 == tail_s2 else 2),
        ])


@lru_cache(maxsize=2**10)
def edit_distance_modify(string1, string2):
    
    if len(string1) == 0: return len(string2)
    if len(string2) == 0: return len(string1)
    
    tail_s1 = string1[-1]
    tail_s2 = string2[-1]
    
    candidates = [
        (edit_distance_modify(string1[:-1], string2) + 1, 'DEL {}'.format(tail_s1)),  # string 1 delete tail
        (edit_distance_modify(string1, string2[:-1]) + 1, 'ADD {}'.format(tail_s2)),  # string 1 add tail of string2
    ]
    
    if tail_s1 == tail_s2:
        both_forward = (edit_distance_modify(string1[:-1], string2[:-1]) + 0, '')
    else:
        both_forward = (edit_distance_modify(string1[:-1], string2[:-1]) + 1, 'SUB {} => {}'.format(tail_s1, tail_s2))

    candidates.append(both_forward)
    
    min_distance, operation = min(candidates, key=lambda x: x[0])
    
    solution[(string1, string2)] = operation 
    
    return min_distance

In [65]:
edit_distance_modify('ABCDE','ABCCEF')

2

In [68]:
solution[('AB','A')]

'DEL B'

In [77]:
# 没有写完，不知什么条件应该是递归的出口
def solution_parse(string1, string2, revenue_solution):
    result = []
    action = revenue_solution[(string1, string2)]
    if len(string1) == len(string2) or abs(len(string1)-len(string2)) == 1:
        return result.append(action)
    
    if action.startswith('ADD'): 
        string2 = string2[:-1]
        
    elif action.startswith('DEL'):
        string1 = string1[:-1]
#         result.append(action)
    else:
        string1 = string1[:-1]
        string2 = string2[:-1]
#         result.append(action)

    return solution_parse(string1, string2, revenue_solution)
  

In [78]:
solution_parse('ABCDE', 'ABCCEF', solution)